In [1]:
import subprocess
import zenhan
import os
import sys
import re
import unicodedata
import argparse
import pandas as pd
import pickle
import json
import tqdm

In [2]:
#企業リストを取得
company_list = pd.read_csv("../data/new2/df_achieve/company_list.csv",index_col=0)
company_list=company_list.drop(14350)
company_list=company_list.drop(52080)
company_list=company_list.drop(95080)
with open('../data/text_data_ver2/non_company_list.pkl', 'rb') as f:
    non_list = pickle.load(f)
for com in non_list:
    company_list=company_list.drop(com)

In [246]:
company=16620
df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
#dfを呼び出し
df = pd.read_csv(df_path,index_col=0)
df

,uriage,pred,achieve,f_score,j_score,word_cnt,pdf_code,title,tuki_pred
2017-08-08,59803,211025.0,-0.716607,-0.017162,0.017162,874,20170804448287,平成30年3月期第1四半期決算短信〔日本基準〕（連結）,211025.0
2017-11-10,112382,211025.0,-0.467447,-0.002296,0.021814,871,20171107410918,平成30年3月期第2四半期決算短信〔日本基準〕（連結）,211025.0
2018-02-09,158274,211025.0,-0.249975,-0.003953,0.027668,759,20180201461875,平成30年3月期第3四半期決算短信〔日本基準〕（連結）,211025.0
2018-05-11,230629,265649.0,0.092899,0.025227,0.032291,1982,20180509430631,平成30年3月期決算短信〔日本基準〕（連結）,265649.0
2018-08-08,57623,265649.0,-0.783086,-0.023077,0.061538,130,20180808494181,平成31年3月期第1四半期決算短信〔日本基準〕（連結）,265649.0
2018-11-09,115136,265649.0,-0.566586,0.023148,0.025463,864,20181107431549,平成31年3月期 第2四半期決算短信〔日本基準〕（連結）,265649.0
2019-02-12,207425,265649.0,-0.219176,-0.026764,0.032847,822,20190123462606,平成31年3月期 第3四半期決算短信〔日本基準〕（連結）,265649.0
2019-05-10,267980,308650.0,0.008775,0.023121,0.030347,2076,20190510420023,2019年3月期 決算短信〔日本基準〕（連結）,308650.0
2019-08-09,93252,308650.0,-0.697871,-0.023988,0.026987,667,20190808485187,2020年3月期 第1四半期決算短信〔日本基準〕（連結）,308650.0
2019-11-08,162346,308650.0,-0.474013,-0.030973,0.029499,678,20191108422329,2020年3月期 第2四半期決算短信〔日本基準〕（連結）,308650.0


In [269]:
non_dic={}
for company in tqdm.tqdm(company_list.index):
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    #dfを呼び出し
    df = pd.read_csv(df_path,index_col=0)

    #スコアを算出
    pos_list=[]
    neg_list=[]
    
    non_dic[company]=[]
    #企業の決算短信を一つ一つ見る
    for i,code in enumerate(df['pdf_code']):
        json_path = '../data/text_data_ver2/text_data_sep/'+str(company)+'/'+str(code)+'.json'
        text_path = '../data/text_data_ver2/text_data/'+str(company)+'/'+str(code)+'.txt'
        pkl_path = '../data/text_data_ver2/text_mokuzi_data_pre/'+str(company)+'/'+str(code)+'.pkl'
        with open(json_path, 'r') as f:
            json_load = json.load(f)
        
        if json_load["text"]==[]:
            with open(text_path,'r') as t:
                text_load = t.read()
            if len(text_load) >100 and "訂正" not in df["title"][i]  and "修正" not in df["title"][i]:
                with open(pkl_path, 'rb') as p:
                    pkl_load=pickle.load(p)
               # if pkl_load!=[]:
                non_dic[company].append(code)

100%|██████████████████████████████████████| 2510/2510 [00:04<00:00, 617.13it/s]


In [272]:
cnt=0
cnt_com=0
com_list=[]
for com in non_dic.keys():
    if non_dic[com]!=[]:
        cnt+=len(non_dic[com])
        #if len(non_dic[com])>3:
            #print(com)
        cnt_com+=1
        com_list.append(com)
            
print(cnt)
print(cnt_com)

455
155


In [273]:
com_list

[16620,
 17360,
 17430,
 17800,
 18120,
 18500,
 18690,
 18880,
 19140,
 19260,
 19280,
 19390,
 19420,
 19460,
 19750,
 21620,
 21690,
 22060,
 22670,
 23090,
 23530,
 24830,
 25900,
 26680,
 26700,
 26760,
 26860,
 27220,
 27370,
 27530,
 27760,
 28040,
 28770,
 29230,
 31100,
 31210,
 31660,
 33220,
 33470,
 34070,
 34820,
 36450,
 37700,
 38510,
 38770,
 39000,
 39250,
 39440,
 39610,
 39910,
 40410,
 40470,
 40950,
 42060,
 42120,
 42340,
 44420,
 45210,
 45340,
 45440,
 45530,
 46670,
 46780,
 46850,
 47610,
 49200,
 49940,
 53950,
 58020,
 59150,
 59420,
 59450,
 59470,
 59640,
 59850,
 60810,
 62550,
 62780,
 62890,
 63090,
 63100,
 63190,
 63660,
 63780,
 64000,
 64330,
 64920,
 65180,
 66180,
 66770,
 67180,
 67440,
 67550,
 68090,
 70210,
 70330,
 70590,
 72110,
 72190,
 72240,
 72360,
 72390,
 72690,
 73990,
 74120,
 74380,
 75020,
 75320,
 75550,
 75590,
 75810,
 76060,
 76810,
 77470,
 78360,
 78590,
 79110,
 79230,
 79920,
 80300,
 80350,
 81050,
 81320,
 81350,
 81520,


In [274]:
#保存
with open('non_company_list.pkl', 'wb') as p:
    pickle.dump(com_list, p)

In [154]:
df["title"][0]

'平成30年3月期\u3000第1四半期決算短信〔日本基準〕（連結）'

In [49]:
def normalize(text):
    #text = re.sub(r'\d+', '0', text)
    text = re.sub(r'（', '(', text)
    text = re.sub(r'）', ')', text)
    text = re.sub('．','.',text)
    text=re.sub('０','0',text)
    text=re.sub('１','1',text)
    text=re.sub('２','2',text)
    text=re.sub('３','3',text)
    text=re.sub('４','4',text)
    text=re.sub('５','5',text)
    text=re.sub('６','6',text)
    text=re.sub('７','7',text)
    text=re.sub('８','8',text)
    text=re.sub('９','9',text)
    return(text)

In [241]:
for company in tqdm.tqdm(non_dic.keys()):
#     company=13320
    if non_dic[company]!=[]:
        for code in non_dic[company]:
            txt_path = '../data/text_data_ver2/text_data/'+str(company)+'/'+str(code)+'.txt'
            out_path = '../data/text_data_ver2/text_data_sep/'+str(company)+'/'+str(code)+'.json'
            pkl_path = '../data/text_data_ver2/text_mokuzi_data_pre/'+str(company)+'/'+str(code)+'.pkl'

            with open(txt_path, 'r') as f:
                txt_load = f.read()

            with open(pkl_path, 'rb') as p:
                pkl_load=pickle.load(p)
        
        
            text_list=[]
            text_dic={"text":[]}
            #kari_dic={}
            text=normalize(txt_load)
#             if len(text)<3000:
#                 continue
            kari_text=text
            mokuzi_list= pkl_load
            true_mokuzi=[]
            false_mokuzi=[]
            try:
                for i,mokuzi in enumerate(mokuzi_list):
                    if len(mokuzi)>3:
                        kari=mokuzi[0:3]
    #                     if "（" in kari or "(" in kari or "（" in kari or kari[0].isdigit() or kari[1].isdigit() or kari[2].isdigit():
                        if '\n' in mokuzi:
                            mokuzi= mokuzi.replace('\n','')
                        mokuzi = normalize(mokuzi)
                        ind=kari_text.find(mokuzi)
                        if ind!=-1:
                            #目次があったものは変数に入れておく
                            true_mokuzi.append(mokuzi)
                            kari_dic={}
                            if text_dic["text"]!=[]:
                                kari_dic["title"]=true_mokuzi[-2]
                                kari_dic["text"]=kari_text[len(true_mokuzi[-2]):ind]
                                text_dic["text"].append(kari_dic)
                                kari_text=kari_text[ind:]

                                #mokuzi_listの最後に来た時の処理
                                if mokuzi==mokuzi_list[-1]:
                                    kari_dic={}
                                    kari_dic["title"]=true_mokuzi[-1]
                                    kari_dic["text"]=kari_text[len(true_mokuzi[-1]):]
                                    text_dic["text"].append(kari_dic)
                                    break

                            #false_mokuziに目次があるとき（textのない目次）
                                if false_mokuzi!=[]:
                                    for non_mokuzi in false_mokuzi:
                                        kari_dic={}
                                        kari_dic["title"]=non_mokuzi
                                        kari_dic["text"]=""
                                        text_dic["text"].append(kari_dic)
                                    false_mokuzi=[]

                            else:
                                kari_dic["title"]="目次より前"
                                kari_dic["text"]=text[:ind]
                                text_dic["text"].append(kari_dic)
                                kari_text=text[ind:]
                                
                                if false_mokuzi!=[]:
                                    for non_mokuzi in false_mokuzi:
                                        kari_dic={}
                                        kari_dic["title"]=non_mokuzi
                                        kari_dic["text"]=""
                                        text_dic["text"].append(kari_dic)
                                    false_mokuzi=[]

                        else:
                            false_mokuzi.append(mokuzi)
                             #mokuzi_listの最後に来た時の処理
                            if mokuzi==mokuzi_list[-1]:
                                kari_dic={}
                                kari_dic["title"]=true_mokuzi[-1]
                                kari_dic["text"]=kari_text[len(true_mokuzi[-1]):]
                                text_dic["text"].append(kari_dic)
                                if false_mokuzi!=[]:
                                    for non_mokuzi in false_mokuzi:
                                        kari_dic={}
                                        kari_dic["title"]=non_mokuzi
                                        kari_dic["text"]=""
                                        text_dic["text"].append(kari_dic)
                                    false_mokuzi=[]
                #保存
                json_file = open(out_path, 'w')
                json.dump(text_dic, json_file, ensure_ascii=False)
                json_file.close()
            except:
                continue
                print(company,code)

100%|█████████████████████████████████████| 2510/2510 [00:00<00:00, 5400.79it/s]


In [198]:
mokuzi_list

['\n', '（４）今後の見通し', '３．連結財務諸表及び主な注記', '（１）連結貸借対照表', '連結損益計算書']

In [203]:
text

'(注)前連結会計期間において営業外収益に表示していた「情報提供料収入」について、当連結会計期間より売上高に組替えて表示しております。\n。\n※四半期決算短信は公認会計士又は監査法人の四半期レビューの対象外です※業績予想の適切な利用に関する説明、その他特記事項上記の予想は、本資料発表日現在において入手可能な情報及び将来の業績に影響を与える不確実な要因に係る本資料発表時現在における仮定を前提としています。\n実際の業績は、今後、様々な要因によって大きく異なる結果となる可能性があります。\n。\n(株)文教堂グループホールディングス(9978)平成30年8月期第2四半期決算短信(3)四半期連結財務諸表に関する注記事項(継続企業の前提に関する注記)該当事項はありません。\n(株主資本の金額に著しい変動があった場合の注記)該当事項はありません。\n。\n'

In [129]:
text_dic

{'text': []}

In [122]:
text[1732:1745]

'(1)当期の経営成績の概況'

In [127]:
text[1732:1745]==normalize(false_mokuzi[1])

True

In [119]:
false_mokuzi[1]

'（１）当期の経営成績の概況'

In [118]:
text.find(false_mokuzi[1])

-1

In [36]:
with open(out_path, 'r') as p:
    load=json.load(p)

In [256]:
for company in tqdm.tqdm(non_dic.keys()):
#      company=45210
    if non_dic[company]!=[]:
        for code in non_dic[company]:
            txt_path = '../data/text_data_ver2/text_data/'+str(company)+'/'+str(code)+'.txt'
            out_path = '../data/text_data_ver2/text_data_sep/'+str(company)+'/'+str(code)+'.json'
            pkl_path = '../data/text_data_ver2/text_mokuzi_data_pre/'+str(company)+'/'+str(code)+'.pkl'

            with open(txt_path, 'r') as f:
                txt_load = f.read()

            with open(pkl_path, 'rb') as p:
                pkl_load=pickle.load(p)
            
            if len(txt_load)>3000:
                print(company,code)
                break

100%|████████████████████████████████████| 2510/2510 [00:00<00:00, 75492.84it/s]

18120 20170726439285
18880 20180503427354
19260 20220512543246
19390 20180419414831
19460 20170620410797
26860 20190329498084
34070 20180510433416
38770 20190430413960
40470 20171025498407
45210 20200715462975
45530 20180507427669
63190 20200401488528
72110 20210423499419
72690 20220425526975
76060 20180501425557
79110 20180501426187
82550 20170628418363
82600 20180326497677
90210 20210117445172
93850 20210709464094
95060 20180417413548
99130 20170704424702


In [264]:
company=18880
code = 20170802446109
txt_path = '../data/text_data_ver2/text_data/'+str(company)+'/'+str(code)+'.txt'
out_path = '../data/text_data_ver2/text_data_sep/'+str(company)+'/'+str(code)+'.json'
pkl_path = '../data/text_data_ver2/text_mokuzi_data_pre/'+str(company)+'/'+str(code)+'.pkl'

with open(txt_path, 'r') as f:
    txt_load = f.read()
with open(pkl_path, 'rb') as p:
    pkl_load=pickle.load(p)

In [265]:
txt_load

'（注）直近に公表されている配当予想からの修正の有無：無（注）当社は平成２９年１０月１日を効力発生日として普通株式１０株につき１株の割合で株式併合を実施する予定であるため、平成３０年３月期（予想）の１株当たり期末配当金につきましては、当該株式併合の影響を考慮した金額を記載しています。\n株式併合考慮前の平成３０年３月期（予想）の１株当たり期末配当金は４円００銭となります。\n。\n（注）直近に公表されている業績予想からの修正の有無：無（注）平成３０年３月期の連結業績予想における１株当たり当期純利益につきましては、株式併合の影響を考慮しております。\n。\n※四半期決算短信は四半期レビューの対象外です※業績予想の適切な利用に関する説明、その他特記事項（将来に関する記述等についてのご注意）本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。\nまた、実際の業績等は様々な要因により大きく異なる可能性があります。\n業績予想の前提となる条件及び業績予想のご利用にあたっての注意事項等については、添付資料Ｐ．２「１．当四半期決算に関する定性的情報（３）連結業績予想などの将来予測情報に関する説明」をご覧ください。\n（株式併合後の配当及び業績予想について）当社は、平成２９年１０月１日を効力発生日として、普通株式１０株につき１株の割合で株式併合を実施する予定です。\nまた、株式併合と同日付で、単元株式数の１，０００株から１００株への変更を併せて実施する予定です。\nなお、株式併合考慮前に換算した平成３０年３月期の配当予想、連結業績予想は以下のとおりとなります。\n１．平成３０年３月期の配当予想期末４円００銭２．平成３０年３月期の連結業績予想１株当たり当期純利益通期１９円３０銭。\n１．当四半期決算に関する定性的情報（１）連結経営成績に関する説明当第１四半期連結累計期間におけるわが国経済は、雇用や所得環境の改善が続くなか、各種政策の効果もあって、景気は緩やかな回復基調で推移しました。\n一方、海外経済の不確実性や米国の政策の動向及び影響等に留意する必要があり、景気の先行きは不透明な状況が続いています。\n建設業界におきましては、公共投資は底堅さが増し

In [266]:
pkl_load

['１.',
 '（',
 '（',
 '（',
 '２',
 '（',
 '（',
 '四半期連結損益計算書',
 '第',
 '四半期連結包括利益計算書',
 '第',
 '（',
 '（',
 '（',
 '３',
 '個']

In [260]:
company=17120
if non_dic[company]!=[]:
    for code in non_dic[company]:
        txt_path = '../data/text_data_ver2/text_data/'+str(company)+'/'+str(code)+'.txt'
        out_path = '../data/text_data_ver2/text_data_sep/'+str(company)+'/'+str(code)+'.json'
        pkl_path = '../data/text_data_ver2/text_mokuzi_data_pre/'+str(company)+'/'+str(code)+'.pkl'

        with open(txt_path, 'r') as f:
            txt_load = f.read()

        with open(pkl_path, 'rb') as p:
            pkl_load=pickle.load(p)


        text_list=[]
        text_dic={"text":[]}
        #kari_dic={}
        text=normalize(txt_load)
        kari_text=text
        mokuzi_list= pkl_load
        true_mokuzi=[]
        false_mokuzi=[]
        for i,mokuzi in enumerate(mokuzi_list):
            if len(mokuzi)>3:
                
                if '\n' in mokuzi:
                    mokuzi= mokuzi.replace('\n','')
                    mokuzi= mokuzi.replace('目次','')
                mokuzi = normalize(mokuzi)
                ind=kari_text.find(mokuzi)
                print(ind)
                if ind!=-1:
                    #目次があったものは変数に入れておく
                    true_mokuzi.append(mokuzi)
                    kari_dic={}
                    if text_dic["text"]!=[]:
                        kari_dic["title"]=true_mokuzi[-2]
                        kari_dic["text"]=kari_text[len(true_mokuzi[-2]):ind]
                        text_dic["text"].append(kari_dic)
                        kari_text=kari_text[ind:]
                 
                        #mokuzi_listの最後に来た時の処理
                        if mokuzi==mokuzi_list[-1]:
                            kari_dic={}
                            kari_dic["title"]=true_mokuzi[-1]
                            kari_dic["text"]=kari_text[len(true_mokuzi[-1]):]
                            text_dic["text"].append(kari_dic)
                            break

                    #false_mokuziに目次があるとき（textのない目次）
                        if false_mokuzi!=[]:
                            for non_mokuzi in false_mokuzi:
                                kari_dic={}
                                kari_dic["title"]=non_mokuzi
                                kari_dic["text"]=""
                                text_dic["text"].append(kari_dic)
                            false_mokuzi=[]

                    else:
                        kari_dic["title"]="目次より前"
                        kari_dic["text"]=text[:ind]
                        text_dic["text"].append(kari_dic)
                        kari_text=text[ind:]
                        
                        if false_mokuzi!=[]:
                            for non_mokuzi in false_mokuzi:
                                kari_dic={}
                                kari_dic["title"]=non_mokuzi
                                kari_dic["text"]=""
                                text_dic["text"].append(kari_dic)
                            false_mokuzi=[]

                else:
                    false_mokuzi.append(mokuzi)
                     #mokuzi_listの最後に来た時の処理
                    if mokuzi==mokuzi_list[-1]:
                        kari_dic={}
                        kari_dic["title"]=true_mokuzi[-1]
                        kari_dic["text"]=kari_text[len(true_mokuzi[-1]):]
                        text_dic["text"].append(kari_dic)
                        if false_mokuzi!=[]:
                            for non_mokuzi in false_mokuzi:
                                kari_dic={}
                                kari_dic["title"]=non_mokuzi
                                kari_dic["text"]=""
                                text_dic["text"].append(kari_dic)
                            false_mokuzi=[]

In [232]:
false_mokuzi

['1.経営成績等の状況',
 '3.連結財務諸表及び主な注記',
 '(1)連結貸借対照表',
 '(2)連結損益計算書及び連結包括利益計算書',
 '連結損益計算書',
 '連結包括利益計算書',
 '(3)連結株主資本等変動計算書',
 '(4)連結キャッシュ・フロー計算書',
 '(セグメント情報)',
 '(1株当たり情報)']

In [224]:
true_mokuzi

['(1)当期の経営成績の概況',
 '(2)当期の財政状態の概況',
 '(4)今後の見通し',
 '2.会計基準の選択に関する基本的な考え方',
 '(5)連結財務諸表に関する注記事項',
 '(継続企業の前提に関する注記)',
 '(追加情報)',
 '(重要な後発事象)']

In [192]:
kari_dic

{}